In [1]:
# Standard
import logging

# Third party
import numpy as np
import rdkit
from rdkit import RDLogger
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader

# GT-PyG
from gt_pyg.data.utils import (
    get_tensor_data, 
    get_node_dim, 
    get_edge_dim, 
    get_train_valid_test_data
)
from gt_pyg.nn.model import GraphTransformerNet
from gt_pyg.util import get_ww_statistic
from gt_pyg.util import get_correction_factor, prepare_params

import weightwatcher as ww

# Turn off majority of RDKit warnings
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)


# Set a random seed for a reproducibility purposes
torch.manual_seed(192837465)

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Log the used versions of RDkit and torch
print(f'Numpy version: {np.__version__}')
print(f'Rdkit version: {rdkit.__version__}')
print(f'Torch version: {torch.__version__}')
print(f'WeightWatcher version: {ww.__version__}')

/Users/pawel/anaconda3/envs/gt/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch is available but CUDA is not. Defaulting to SciPy for SVD


Numpy version: 1.21.6
Rdkit version: 2022.09.5
Torch version: 1.13.1
WeightWatcher version: 0.7.1.5


## Get the ADME@TDC data

**Note**: To use the code below, make sure that the chosen endpoint is a regression task.

In [2]:
from tdc import utils
names = utils.retrieve_benchmark_names('ADMET_Group')
output = "\n".join([f"{index}. {name}" for index, name in enumerate(names, start=1)])
print("Available endpoints:\n\n" + output)

Available endpoints:

1. caco2_wang
2. hia_hou
3. pgp_broccatelli
4. bioavailability_ma
5. lipophilicity_astrazeneca
6. solubility_aqsoldb
7. bbb_martins
8. ppbr_az
9. vdss_lombardo
10. cyp2d6_veith
11. cyp3a4_veith
12. cyp2c9_veith
13. cyp2d6_substrate_carbonmangels
14. cyp3a4_substrate_carbonmangels
15. cyp2c9_substrate_carbonmangels
16. half_life_obach
17. clearance_microsome_az
18. clearance_hepatocyte_az
19. herg
20. ames
21. dili
22. ld50_zhu


Regression endpoints with MAE metric:
1. caco2_wang (Best: 0.285 ± 0.005)
2. lipophilicity_astrazeneca (Best: 0.535 ± 0.012)
3. solubility_aqsoldb (Best: 0.776 ± 0.008)
4. ppbr_az (Best: 9.185 ± 0.000)
5. ld50_zhu (Best: 0.588 ± 0.005)

In [3]:
PE_DIM = 6
(tr, va, te) = get_train_valid_test_data('caco2_wang', min_num_atoms=0)
tr_dataset = get_tensor_data(tr.Drug.to_list(), tr.Y.to_list(), pe_dim=PE_DIM)
va_dataset = get_tensor_data(va.Drug.to_list(), va.Y.to_list(), pe_dim=PE_DIM)
te_dataset = get_tensor_data(te.Drug.to_list(), te.Y.to_list(), pe_dim=PE_DIM)
NODE_DIM = get_node_dim()
EDGE_DIM = get_edge_dim()

print(f'Number of training examples: {len(tr_dataset)}')
print(f'Number of validation examples: {len(va_dataset)}')
print(f'Number of test examples: {len(te_dataset)}')

train_loader = DataLoader(tr_dataset, batch_size=128)
val_loader = DataLoader(va_dataset, batch_size=512)
test_loader = DataLoader(te_dataset, batch_size=512)

Found local copy...
Loading...
Done!


Number of training examples: 637
Number of validation examples: 91
Number of test examples: 182


## Train and eval the GT model

### Auxiliary functions

In [4]:
def train(epoch, loss_func, warmup_epochs=5):
    model.train()
    
    
    if epoch > warmup_epochs:
        corrections = get_correction_factor(model, optimizer)
        for idx, val in enumerate(corrections):
            optimizer.param_groups[idx]['lr'] = val * optimizer.param_groups[idx]['base_lr']
            
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        # randomly flip sign of eigenvectors
        batch_pe = data.pe * (2 * torch.randint(low=0, high=2, size=(1, PE_DIM)).float() - 1.0)
        (out,_) = model(data.x, data.edge_index, data.edge_attr, batch_pe, data.batch, zero_var=False)
        loss = loss_func(out.squeeze(), data.y)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, loss_func):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        # randomly flip sign of eigenvectors
        batch_pe = data.pe * (2 * torch.randint(low=0, high=2, size=(1, PE_DIM)).float() - 1.0)
        (out,_) = model(data.x, data.edge_index, data.edge_attr, batch_pe, data.batch)
        total_error += loss_func(out.squeeze(), data.y).item()
    return total_error / len(loader.dataset)

train_loss = nn.L1Loss(reduction='mean')
test_loss = nn.L1Loss(reduction='sum')

### Slighlty optimized Graph Transformer architecture

1. `gelu` activation is used instead of `relu`
2. Multiaggregator used for global pooling
3. Multiaggregator used for message passing

Number of params 873k instead of 701k

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphTransformerNet(node_dim_in=NODE_DIM,
                            edge_dim_in=EDGE_DIM,
                            pe_in_dim=PE_DIM,
                            num_gt_layers=7, 
                            hidden_dim=128,
                            num_heads=8,
                            norm='bn',
                            gt_aggregators=['sum', 'mean'],
                            aggregators=['sum','mean','max', 'std'],
                            dropout=0.1,
                            act='relu').to(device)

if int(torch.__version__.split('.')[0]) >= 2:
    model = torch_geometric.compile(model) 

#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

lr = 0.001
params = prepare_params(model, lr)
optimizer = torch.optim.AdamW(params)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10,
                              min_lr=0.00001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10,
                              min_lr=0.00001)

print(model)
print(f"Number of params: {model.num_parameters()//1000} k")


best_epoch = 0
best_validation_loss = np.inf
test_set_mae = np.inf

epochs_list = []
train_loss_list = []
valid_loss_list = []
test_loss_list = []
ww_list = []

EPOCHS=25
for epoch in range(1, EPOCHS + 1):
    tr_loss = train(epoch, loss_func=train_loss, warmup_epochs=2)
    va_loss = test(val_loader, loss_func=test_loss)
    te_loss = test(test_loader, loss_func=test_loss)
    scheduler.step(va_loss)
    print(f'Epoch: {epoch:02d}, Loss: {tr_loss:.4f}, Val: {va_loss:.4f}, '
          f'Test: {te_loss:.4f}')
    if va_loss < best_validation_loss:
        best_epoch = epoch
        best_validation_loss = va_loss
        test_set_mae = te_loss
    
    epochs_list.append(epoch)
    train_loss_list.append(tr_loss)
    valid_loss_list.append(va_loss)
    test_loss_list.append(te_loss)
    ww_list.append(get_ww_statistic(model))
    
print("\nModel's performance on the test set\n"
        "===================================\n"
        f'MAE={test_set_mae}\n'
        f'Epoch={best_epoch}')

GraphTransformerNet(
  (node_emb): Linear(in_features=76, out_features=128, bias=False)
  (edge_emb): Linear(in_features=10, out_features=128, bias=False)
  (pe_emb): Linear(in_features=6, out_features=128, bias=False)
  (gt_layers): ModuleList(
    (0): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (1): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (2): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (3): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (4): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (5): GTConv(128, 128, heads=8, aggrs: sum,mean)
    (6): GTConv(128, 128, heads=8, aggrs: sum,mean)
  )
  (global_pool): MultiAggregation([
    SumAggregation(),
    MeanAggregation(),
    MaxAggregation(),
    StdAggregation(),
  ], mode=cat)
  (mu_mlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (log_var_mlp): MLP(
    (mlp): Sequential(
      (

In [ ]:
plt.scatter(test_loss_list, ww_list, label='WW statistic')
plt.legend(loc=0)

In [ ]:
plt.scatter(test_loss_list, valid_loss_list, label='Valid loss')
plt.scatter(test_loss_list, train_loss_list, label='Train loss')
plt.legend(loc=0)

In [ ]:
test_loss_list[np.argmin(ww_list)]

In [ ]:
test_loss_list[np.argmin(valid_loss_list)]

In [ ]:
test_loss_list[np.argmin(train_loss_list)]

In [ ]:
#plt.plot(epochs_list, train_loss_list)
#plt.plot(epochs_list, valid_loss_list)
#plt.plot(epochs_list, test_loss_list)
plt.plot(epochs_list, ww_list)

In [7]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze(plot=False)
details

,layer_id,name,D,M,N,Q,alpha,alpha_weighted,entropy,has_esd,...,rf,sigma,spectral_norm,stable_rank,status,sv_max,warning,weak_rank_loss,xmax,xmin
0,1,Linear,0.113996,76,1024,13.473684,16.097953,7.829635,0.991447,True,...,1,4.774392,3.064606,45.963947,success,1.750602,under-trained,0,3.064606,2.529707
1,2,Linear,0.194484,10,1024,102.400000,10.787483,3.886509,0.998182,True,...,1,3.460398,2.292342,8.748985,success,1.514048,under-trained,0,2.292342,1.866512
2,11,Linear,0.124354,1024,1024,1.000000,13.124620,7.893820,0.927745,True,...,1,1.993275,3.994398,257.099425,success,1.998599,under-trained,2,3.994398,3.270787
3,12,Linear,0.127330,1024,1024,1.000000,17.020088,10.216418,0.927802,True,...,1,3.083067,3.983423,258.221173,success,1.995852,under-trained,0,3.983423,3.423457
4,13,Linear,0.124529,1024,1024,1.000000,14.741762,8.975403,0.927685,True,...,1,2.392134,4.062954,252.492299,success,2.015677,under-trained,2,4.062954,3.344246
5,14,Linear,0.093337,1024,2048,2.000000,25.902872,16.502720,0.963767,True,...,1,6.429894,4.336107,316.104104,success,2.082332,under-trained,0,4.336107,3.613455
6,15,Linear,0.101943,1024,1024,1.000000,35.168939,21.544822,0.927763,True,...,1,11.389646,4.098353,250.424878,success,2.024439,under-trained,1,4.098353,3.732165
7,16,Linear,0.126739,1024,1024,1.000000,7.855527,4.730978,0.927905,True,...,1,0.761725,4.001735,256.991851,success,2.000434,under-trained,1,4.001735,2.815774
8,19,Linear,0.079860,1024,1024,1.000000,10.768013,3.333208,0.927123,True,...,1,1.525507,2.039610,169.150866,success,1.428149,under-trained,1,2.039610,1.087692
9,22,Linear,0.106462,1024,1024,1.000000,26.708461,4.165708,0.927651,True,...,1,8.129729,1.432088,240.334707,success,1.196699,under-trained,1,1.432088,1.254787


In [8]:
watcher.get_summary(details)

{'log_norm': 2.7709467323522996,
 'alpha': 18.66897386077971,
 'alpha_weighted': 8.048410570823066,
 'log_alpha_norm': 8.663418134741027,
 'log_spectral_norm': 0.45470587308592203,
 'stable_rank': 233.03134457881845}